In [1]:
import string

import aesara
import aesara.sparse
import aesara.tensor as aet
import numpy as np
import pandas as pd
import scipy.sparse as sp

from tabmat import CategoricalMatrix

import sparsedot

The `sparsedot` module implements functions for the computation of the dot product between a sparse matrix $X$ and a column vector $y$. These functions are implemented using the NumPy C-API. 

This module contains the following functions:

* `matrix_vector()`: Applies for the most general case where the $X$ is a sparse matrix of any type.
* `binary_matrix_vector()`: Applies when $X$ is a binary sparse matrix, i.e. a matrix that contains 0s and 1s.
* `single_binary_matrix_vector()`: Applies when $X$ is a binary sparse matrix that contain one, and only one, 1 value per row.

In [2]:
# The "categorical variables"
strings = list(string.ascii_lowercase) + list(string.ascii_uppercase)
strings += [s * 2 for s in strings]
len(strings)

104

In [3]:
class Matrix:
    def __init__(self, X):
        X_sp = sp.csr_matrix(X)
        self.offset = X_sp.indptr
        self.column = X_sp.indices
        self.value = X_sp.data
        self.length = X.shape[0]
    
    def dot(self, other):
        result = np.zeros(self.length, dtype=other.dtype)
        sparsedot.matrix_vector(self.offset, self.column, self.value, other, result)
        return result


class BinaryMatrix:
    def __init__(self, X):
        self.length = X.shape[0]
        self.X_indices = np.ascontiguousarray(np.vstack(np.where(X != 0)).T).astype(np.int32)
        
    def dot(self, other):
        result = np.zeros(self.length, dtype=other.dtype)
        sparsedot.binary_matrix_vector(self.X_indices, other, result)
        return result


class SingleBinaryMatrix:
    def __init__(self, X):
        self.length = X.shape[0]
        self.indices = np.ascontiguousarray(np.where(X != 0)[1]).astype(np.int32)
    
    def dot(self, other):
        result = np.zeros(self.length, dtype=other.dtype)
        sparsedot.single_binary_matrix_vector(self.indices, other, result)
        return result


In [4]:
y = np.arange(len(strings), dtype=np.float64)

In [5]:
x = np.random.choice(strings, size=1000)
X = np.asarray(pd.get_dummies(x))
X_sp = sp.csr_matrix(X)
matrix = Matrix(X)
binary_matrix = BinaryMatrix(X)
single_binary_matrix = SingleBinaryMatrix(X)
tbmat = CategoricalMatrix(x)

Defining Aesara stuff...

In [6]:
aet_x = aet.dmatrix("x")
aet_y = aet.dvector("y")
aet_Y = aet.dmatrix("y")

x_sparse = aesara.sparse.CSR(X_sp.data, X_sp.indices, X_sp.indptr, X_sp.shape)

aet_dot = aesara.function([aet_x, aet_y], aet.dot(aet_x, aet_y))
aet_sparse_dot = aesara.function([x_sparse, aet_Y], aesara.sparse.structured_dot(x_sparse, aet_Y))

In [7]:
print("Matrix")
%timeit matrix.dot(y)

print("\nBinary Matrix")
%timeit binary_matrix.dot(y)

print("\nSingle Binary Matrix")
%timeit single_binary_matrix.dot(y)

print("\nNumPy dense")
%timeit X.dot(y)

print("\nSciPy Sparse")
%timeit X_sp.dot(y)

print("\ntabmat CategoricalMatrix")
%timeit tbmat.matvec(y)

print("\nAesara dot")
%timeit aet_dot(X, y)

print("\nAesara sparse dot")
%timeit aet_sparse_dot(X_sp, y[:, None])

Matrix
3.36 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Binary Matrix
2.24 µs ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Single Binary Matrix
1.98 µs ± 58.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

NumPy dense
81.8 µs ± 9.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

SciPy Sparse
5.93 µs ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

tabmat CategoricalMatrix
8.23 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Aesara dot
101 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Aesara sparse dot
36 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
x = np.random.choice(strings, size=10_000)
X = np.asarray(pd.get_dummies(x))
X_sp = sp.csr_matrix(X)
matrix = Matrix(X)
binary_matrix = BinaryMatrix(X)
single_binary_matrix = SingleBinaryMatrix(X)
tbmat = CategoricalMatrix(x)

print("Matrix")
%timeit matrix.dot(y)

print("\nBinary Matrix")
%timeit binary_matrix.dot(y)

print("\nSingle Binary Matrix")
%timeit single_binary_matrix.dot(y)

print("\nNumPy dense")
%timeit X.dot(y)

print("\nSciPy Sparse")
%timeit X_sp.dot(y)

print("\ntabmat CategoricalMatrix")
%timeit tbmat.matvec(y)

print("\nAesara dot")
%timeit aet_dot(X, y)

print("\nAesara sparse dot")
%timeit aet_sparse_dot(X_sp, y[:, None])

Matrix
15.3 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Binary Matrix
11.3 µs ± 85.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Single Binary Matrix
7.87 µs ± 55.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

NumPy dense
1.38 ms ± 66.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

SciPy Sparse
21.2 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

tabmat CategoricalMatrix
13.5 µs ± 650 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Aesara dot
1.71 ms ± 19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Aesara sparse dot
52.4 µs ± 527 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
x = np.random.choice(strings, size=100_000)
X = np.asarray(pd.get_dummies(x))
X_sp = sp.csr_matrix(X)
matrix = Matrix(X)
binary_matrix = BinaryMatrix(X)
single_binary_matrix = SingleBinaryMatrix(X)
tbmat = CategoricalMatrix(x)

print("Matrix")
%timeit matrix.dot(y)

print("\nBinary Matrix")
%timeit binary_matrix.dot(y)

print("\nSingle Binary Matrix")
%timeit single_binary_matrix.dot(y)

print("\nNumPy dense")
%timeit X.dot(y)

print("\nSciPy Sparse")
%timeit X_sp.dot(y)

print("\ntabmat CategoricalMatrix")
%timeit tbmat.matvec(y)

print("\nAesara dot")
%timeit aet_dot(X, y)

print("\nAesara sparse dot")
%timeit aet_sparse_dot(X_sp, y[:, None])

Matrix
141 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Binary Matrix
101 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Single Binary Matrix
67.6 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

NumPy dense
19.9 ms ± 555 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

SciPy Sparse
177 µs ± 927 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

tabmat CategoricalMatrix
51.3 µs ± 4.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Aesara dot
23.1 ms ± 892 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Aesara sparse dot
237 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
x = np.random.choice(strings, size=1_000_000)
X = np.asarray(pd.get_dummies(x))
X_sp = sp.csr_matrix(X)
matrix = Matrix(X)
binary_matrix = BinaryMatrix(X)
single_binary_matrix = SingleBinaryMatrix(X)
tbmat = CategoricalMatrix(x)


print("Matrix")
%timeit matrix.dot(y)

print("\nBinary Matrix")
%timeit binary_matrix.dot(y)

print("\nSingle Binary Matrix")
%timeit single_binary_matrix.dot(y)

print("\nNumPy dense")
%timeit X.dot(y)

print("\nSciPy Sparse")
%timeit X_sp.dot(y)

print("\ntabmat CategoricalMatrix")
%timeit tbmat.matvec(y)

print("\nAesara dot")
%timeit aet_dot(X, y)

print("\nAesara sparse dot")
%timeit aet_sparse_dot(X_sp, y[:, None])

Matrix
3.33 ms ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Binary Matrix
2.3 ms ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Single Binary Matrix
1.98 ms ± 5.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

NumPy dense
192 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

SciPy Sparse
3.3 ms ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

tabmat CategoricalMatrix
1.99 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Aesara dot
278 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Aesara sparse dot
3.41 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
x = np.random.choice(strings, size=10_000_000)
X = np.asarray(pd.get_dummies(x))
X_sp = sp.csr_matrix(X)
matrix = Matrix(X)
binary_matrix = BinaryMatrix(X)
single_binary_matrix = SingleBinaryMatrix(X)
tbmat = CategoricalMatrix(x)


print("Matrix")
%timeit matrix.dot(y)

print("\nBinary Matrix")
%timeit binary_matrix.dot(y)

print("\nSingle Binary Matrix")
%timeit single_binary_matrix.dot(y)

print("\nNumPy dense")
%timeit X.dot(y)

print("\nSciPy Sparse")
%timeit X_sp.dot(y)

print("\ntabmat CategoricalMatrix")
%timeit tbmat.matvec(y)

print("\nAesara dot")
%timeit aet_dot(X, y)

print("\nAesara sparse dot")
%timeit aet_sparse_dot(X_sp, y[:, None])

Matrix
62.9 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Binary Matrix
46.2 ms ± 961 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Single Binary Matrix
32.6 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

NumPy dense
1.94 s ± 41.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

SciPy Sparse
65.5 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

tabmat CategoricalMatrix
33.3 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Aesara dot
2.68 s ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Aesara sparse dot
62.6 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
